In [ ]:
#@markdown
#Librería para manipular datos
import pandas as pd
# llamar a la API de Zendesk
import numpy as np
import requests
import httplib2
import time
import json
import base64
import glob
import os
from os import walk
from urllib import parse
import math

#from google.colab import drive
#drive.mount('/content/drive')

#config
pd.set_option('display.max_columns', None)
#@markdown #***INGRESO DE USUARIO***
Dominio = '' #@param {type: "string"}
Email = '' #@param {type:"string"}
Token = ''#@param {type:"string"}

z_domain= Dominio
email= Email
contraseña= Token


from tqdm import tqdm
from time import sleep

#@markdown
authentication=email+"/token:"+contraseña
message_bytes = authentication.encode('ascii')
base64_bytes=base64.b64encode(message_bytes).decode('ascii')

headers = {'Authorization': 'Basic '+base64_bytes, 'Content-Type': 'application/json'}
start_time="946684800"

In [ ]:

.0
#@markdown
qw= True
all_df=pd.DataFrame()
url = z_domain+"/api/v2/triggers/"
while(qw):
  (url)
  response= requests.get(url, headers=headers)
  respuesta = response.text


 
  data=json.loads(response.text)
  print(data["triggers"])

  url=data['next_page']
  df=pd.DataFrame(data['triggers'])
  all_df=pd.concat([all_df,df[['id','created_at','updated_at','title','description','active','category_id','position','conditions','actions']]])
  time.sleep(5)
  try:
    count=math.ceil(data['count']/100)
    num=np.array(parse.parse_qs(parse.urlsplit(url).query)['page'])
    if count>=int(num):
      print("Valido",count,"Es mayor que",int(num))
      qw= True
    else:
      qw=False
  except:
    qw= False


all_df=all_df.reset_index(drop=True)


In [ ]:
def generarDataSetFormularios(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_forms"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    formularios={}
    for idx, x in enumerate(data['ticket_forms']):
        id = data['ticket_forms'][idx]['id'];
        nombre = data['ticket_forms'][idx]['name'];
        formularios[id] = nombre
        
    return formularios

In [ ]:
def generarDataSetCamposTicket(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_fields"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    campos_de_ticket={}
    for idx, x in enumerate(data['ticket_fields']):
        id =  data['ticket_fields'][idx]['id']
        nombreCampo =  data['ticket_fields'][idx]['title']
        type =  data['ticket_fields'][idx]['type']


        if type == 'tagger' or type == 'multiselect':
            custom_field_options = data['ticket_fields'][idx]['custom_field_options']
            opciones_campo = []
            for idy, y in enumerate(custom_field_options):
                opciones_campo.append([custom_field_options[idy]['raw_name'], custom_field_options[idy]['value']])

            campos_de_ticket[id]=[nombreCampo, type, opciones_campo]

        if type == 'checkbox':
            tag = custom_field_options = data['ticket_fields'][idx]['tag']
            campos_de_ticket[id]=[nombreCampo, type, tag]

        if type != 'checkbox' and type != 'tagger' and type != 'multiselect':
            campos_de_ticket[id]=[nombreCampo, type]

    
    return campos_de_ticket

In [ ]:
def generar_categoria_disparadores(z_domain,start_time,headers):    
    url3 = z_domain+"/api/v2/trigger_categories"
    r=requests.get(url3, headers=headers)
    
    data=json.loads(r.text)
    (data)
    dis_cat={}
    for row in data['trigger_categories']:
      #(row)
      dis_cat[row['id']]=row['name']

    

    
    return dis_cat

In [ ]:
def data_user(z_domain,start_time,headers):
  continua2=True
  continua=True
  org_time=str(946684800)
  user_time=str(946684800)
  all_org={}
  iter=1
  #print("---Lo lamentamos, esto puede demorar")
  while continua2:
    #print("---extración de info org iter "+str(iter)+",COD "+org_time)
    url2 = z_domain+"/api/v2/incremental/organizations.json?start_time="+org_time
    http = httplib2.Http()
    response, content = http.request(url2, 'GET', headers=headers)
    data2=json.loads(content)
    time.sleep(15)
    #(data2)
    #("_________")
    for idx, x in enumerate(data2['organizations']):
      all_org[str(x['id'])]=[x['name']]

    if(data2['end_of_stream']):
      continua2=False 
    org_time=str(data2['end_time'])
  #print("---Se obtuvo la información de las organizaciones")
  #__________________________________________________________________________________________________
  all_user={}
  iter=1
  while continua:
    print("---extración de info usuario iter "+str(iter)+",COD "+user_time)
    url = z_domain+"/api/v2/incremental/users.json?start_time="+user_time
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(15)
    #(data)
    ("_________")
    for idx, x in enumerate(data['users']):
      if x['organization_id'] is not None:
        nom_org=all_org[str(x['organization_id'])][0]
      else:
        nom_org=""
      all_user[str(x['id'])]=[x['name'],x['email'],str(nom_org)]

    if(data['end_of_stream']):
      continua=False 
    user_time=str(data['end_time'])
    iter=iter+1
  #print("---Se obtuvo la información de los usuarios")

  return all_user

In [ ]:
lista_formulario=generarDataSetFormularios(z_domain,start_time,headers)
lista_campos=generarDataSetCamposTicket(z_domain,start_time,headers)
lista_categoría_a=generar_categoria_disparadores(z_domain,start_time,headers)
lista_usuario=data_user(z_domain,start_time,headers)




In [ ]:
#df_y=pd.DataFrame(columns=['Ticket es','form','Centro comercial','¿Ticket hijo?','Categoría','Subcategoría','Tipo disparador','cambia el estado','¿Objetivo?','Asignado','Seguidor asignado 1','Seguidor asignado 2','Seguidor asignado 3','Seguidor asignado 4','Seguidor asignado 5','Seguidor asignado 6'])
Operadores_s={
"is":"Es",
"includes":"Incluye",               
"not_includes":"No incluye",             
"is_not":"No es",                    
"present":"Presente",                                     
"changed":"Cambio a",                
"value_previous":"Valor anterior",           
"not_present":"No esta presente",            
"less_than":"Menor que",
"value":"Valor",               
"greater_than":"Mayor que",              
"not_changed":"No cambio",               
"not_value_previous":"No tiene valores previos", 
} 

basic_parameter={
"notification_user":"Notificar al usuario",
"follower":"Seguidor",
"ticket_form_id":"Formulario",
"update_type":"Ticket es",
"current_tags":"Etiquetas",
"assignee_id":"Agente asignado",
"status":"Estado",
"notification_target":"Notificar a objetivo",
"remove_tags":"Quitar etiquetas",
"notification_webhook":"Notificar a WEBHOOK",
"comment_is_public":"Comentario es público",
"priority":"Prioridad",
"role":"Rol",
"requester_id":"Solicitante es",
"group_id":"Grupo es",
"collaboration_thread":"Colaborador",
"notification_group":"Notificación a grupo",
"ticket_is_public":"Comentario público",
"requester_role":"Rol del solicitante",
"comment_includes_word":"Comentario incluye alguna de las siguientes palabras",
"via_id":"Canal",
"current_via_id":"Canal actual",
"reopens":"Reabrir",
"set_tags":"Reiniciar etiquetas"
}

In [ ]:
print(lista_usuario)

def procesar_parametro(parametro,valor,formularios):
  
  return parametro


In [ ]:
df_y=pd.DataFrame()
df_tran=all_df
formularios = lista_formulario
usuarios = lista_usuario
print(usuarios)

for index, row in df_tran.iterrows():
  #Campos vacios 
  x=row['conditions']
  for y in x["all"]:
    #print(y)
    
    #Validar el parametro
    if "custom" in y['field']:
      param=lista_campos[int(y['field'].split("custom_fields_")[1])][0]
    else:
      param=basic_parameter[y['field']]
    #Validar el operador
    if y.get("operator") is not None:
      Operador=Operadores_s[y['operator']]
    else:
      Operador=""
    
    df_y=df_y.append(pd.Series(    
    {
        "id":row["id"],
        "Nombre disparador":row["title"],
        "descripcion":row["description"],
        "Activo":row["active"],
        "Categogía":lista_categoría_a[row['category_id']],
        "Tipo":"Condición Obligatoria",
        "Parametro":param,
        "Operador":Operador,
        "Valor":y["value"]
    }),ignore_index=True)
  for y in x["any"]:
    #print(y)

    #Validar el parametro
    if "custom" in y['field']:
      param=lista_campos[int(y['field'].split("custom_fields_")[1])][0]
    else:
      param=basic_parameter[y['field']]
    #Validar el operador
    if y.get("operator") is not None:
      Operador=Operadores_s[y['operator']]
    else:
      Operador=""
    df_y=df_y.append(pd.Series(    
    {
        "id":row["id"],
        "Nombre disparador":row["title"],
        "descripcion":row["description"],
        "Activo":row["active"],
        "Categogía":lista_categoría_a[row['category_id']],
        "Tipo":"Condición opcional",
        "Parametro":param,
        "Operador":Operador,
        "Valor":y["value"]
    }),ignore_index=True)
  for y in row["actions"]:
    #print(y)

    #Validar el parametro
    if "custom" in y['field']:
      param=lista_campos[int(y['field'].split("custom_fields_")[1])][0]
    else:
      param=basic_parameter[y['field']]
    #Validar el operador
    if y.get("operator") is not None:
      Operador=Operadores_s[y['operator']]
    else:
      Operador=""
    df_y=df_y.append(pd.Series(    
    {
        "id":row["id"],
        "Nombre disparador":row["title"],
        "descripcion":row["description"],
        "Activo":row["active"],
        "Categogía":lista_categoría_a[row['category_id']],
        "Tipo":"Acciones",
        "Parametro":param,
        "Operador":Operador,
        "Valor":y["value"]
    }),ignore_index=True)


In [ ]:
df_y


In [ ]:
df_y.to_excel('archivo.xlsx', sheet_name='Your sheet name', index=False)

In [ ]:

df_tran=all_df

for index, row in df_tran.iterrows():
  #print("-Se trabajará disparador:",row['id'],"Con nombre",row['title'])

  x=row['conditions']
  cate_disp=lista_categoría_a[row['category_id']]
  
  var_update_type=None
  var_form=None
  var_mall=None
  var_childt=False
  var_category_1lv=None
  var_category_2lv= None
  var_accion_estado="Ninguno"
  var_target=False
  var_follower_0=None
  var_follower_1=None
  var_follower_2=None
  var_follower_3=None
  var_follower_4=None
  var_follower_5=None
  var_assigned=None

  for y in row["actions"]:
    #(y)
    if y['field'] == "status" and y['value'] =="open": var_accion_estado="Abierto"
    if y['field'] == "status" and y['value'] =="new": var_accion_estado="Nuevo"
    if y['field'] == "status" and y['value'] =="pending": var_accion_estado="Pendiente"
    if y['field'] == "status" and y['value'] =="solved": var_accion_estado="Resuelto"
    if y['field'] == "status" and y['value'] =="closed": var_accion_estado="Cerrado"
    if y['field'] == "notification_target": var_target=True
    if y['field'] == "assignee_id":
      
      if y['value'].isnumeric():
        var_assigned=lista_usuario[y['value']][1]
      if isinstance(y['value'], str) and y['value']=="current_user":
        var_assigned=y['value']
    if y['field'] == "follower" and var_follower_0 != None  and var_follower_1 and var_follower_2  and var_follower_3 != None and var_follower_4 != None and var_follower_5 == None: 
      var_follower_5=lista_usuario[y['value']][1] 
      
    if y['field'] == "follower" and var_follower_0 != None  and var_follower_1 and var_follower_2  and var_follower_3 != None and var_follower_4 == None: 
      var_follower_4=lista_usuario[y['value']][1] 

    if y['field'] == "follower" and var_follower_0 != None  and var_follower_1 and var_follower_2 != None and var_follower_3 == None:  
      var_follower_3=lista_usuario[y['value']][1] 

    if y['field'] == "follower" and var_follower_0 != None  and var_follower_1 != None and var_follower_2 == None: 
      var_follower_2=lista_usuario[y['value']][1]

    if y['field'] == "follower" and var_follower_0 != None and var_follower_1 == None: 
      var_follower_1=lista_usuario[y['value']][1] 

    if y['field'] == "follower" and var_follower_0 == None: 
      var_follower_0=lista_usuario[y['value']][1]
    
    
    
    
    
  #print("-- El ticket cambia al estado",var_accion_estado)
  #print("-- ¿Se notifica a un objetivo?",var_target)
  #print("-- Agente asignado",var_assigned)
  #print("-- Seguidor 1",var_follower_0)
  #print("-- Seguidor 2",var_follower_1)
  #print("-- Seguidor 3",var_follower_2)
  #print("-- Seguidor 4",var_follower_3)
  #print("-- Seguidor 5",var_follower_4)
  #print("-- Seguidor 6",var_follower_5)

  for y in x["all"]:
    #(y)
    #Ticket es creado/Actualizado
    if y['field'] == "update_type" and y['value'] =="Create": var_update_type="Creado"
    if y['field'] == "update_type" and y['value'] =="Change": var_update_type="Actualizado"
    
    # Formulario
    if y['field'] == "ticket_form_id" and  y['operator'] =="is": var_form=lista_formulario[int(y['value'])]
    
    # Centro comercial var_mall
    if y['field'] == "custom_fields_360030126932" and  y['operator'] =="is": var_mall=y['value']
    
    # ¿ticket hijo? {'field': 'current_tags', 'operator': 'not_includes', 'value': 'project_child'}
    if y['field'] == "current_tags" and  y['operator'] =="includes" and  y['value'] =="project_child": var_childt=True
    
  #print("-- Disparador se activa cuando",var_update_type)
  #print("-- Considera al Formulario",var_form)
  #print("-- Usa el centro comercial",var_mall)
  #print("-- ¿Considera ticket hijo?",var_childt)
  if var_form=="Security":
    for Z in x["all"]:
      if Z['field'] == "custom_fields_360030127112":
        for v in lista_campos[360030127112][2]:
          if v[1]==Z['value'] and var_category_1lv == None:
            var_category_1lv=v[0]
            #print("-- Se encontró el valor categoría SAFETY:",var_category_1lv)
    if var_category_1lv=="Indemnización":
      var_category_2lv="Denuncia"
      #print("--- Solo puede ser Denuncia la subcategoría")
  
  if var_form=="Safety":
    for Z in x["all"]:
      if Z['field'] == "custom_fields_360030209911":
        for v in lista_campos[360030209911][2]:
          if v[1]==Z['value'] and var_category_1lv == None:
            var_category_1lv=v[0]
            #print("-- Se encontró el valor categoría SAFETY:",var_category_1lv)
    if var_category_1lv=="Accidente":
      var_category_2lv="Denuncia"
      #print("--- Accidente no tiene subcategoría")
    if var_category_1lv=="Atención de Salud":
      for Z in x["all"]:
        if Z['field'] == "custom_fields_4416623654164":
          for v in lista_campos[4416623654164][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              #print("--- Se encontró el valor para Razón de la atenciónActivo:",var_category_2lv)
    if var_category_1lv=="Condición Insegura":
      for Z in x["all"]:
        if Z['field'] == "custom_fields_360030209731":
          for v in lista_campos[360030209731][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              #print("--- Se encontró el valor para Razón de la atenciónActivo:",var_category_2lv)
  
  if var_form=="SAC":
    for Z in x["all"]:
      # Busco si esto es la categoría del formulario de SAC
      if Z['field'] == "custom_fields_360030127172":
        for v in lista_campos[360030127172][2]:
          if v[1]==Z['value'] and var_category_1lv == None:
            var_category_1lv=v[0]
            #print("-- Se encontró el valor categoría SAC:",var_category_1lv)
      
    if var_category_1lv=="Constancia":
      for Z in x["all"]:

        if Z['field'] == "custom_fields_360030127612":
          for v in lista_campos[360030127612][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              #print("--- Se encontró el valor para subcategoría Constancia:",var_category_2lv)
        
        
    if var_category_1lv=="Indemnización":
      var_category_2lv=""
      #print("--- Indemnización no tiene subcategoría")

    if var_category_1lv=="Sugerencia":
      for Z in x["all"]:

        if Z['field'] == "custom_fields_360030127472":
          for v in lista_campos[360030127472][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              ##print("--- Se encontró el valor para subcategoría Sugerencia:",var_category_2lv)
        
    if var_category_1lv=="Reclamo":
      for Z in x["all"]:

        if Z['field'] == "custom_fields_360030209371":
          for v in lista_campos[360030209371][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              ##print("--- Se encontró el valor para subcateogría Reclamo:",var_category_2lv)

    if var_category_1lv=="Felicitación":
      for Z in x["all"]:

        if Z['field'] == "custom_fields_360030209451":
          for v in lista_campos[360030209451][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              #print("--- Se encontró el valor para subcateogría Felicitación:",var_category_2lv)
    
    if var_category_1lv=="Consulta":
      for Z in x["all"]:

        if Z['field'] == "custom_fields_360030127072":
          for v in lista_campos[360030127072][2]:
            if v[1]==Z['value'] and var_category_2lv== None:
              var_category_2lv=v[0]
              #print("--- Se encontró el valor para subcateogría Consulta:",var_category_2lv)
    


  df_y=df_y.append(pd.Series(
    {
      'Ticket es':var_update_type,
     'form':var_form,
     'Centro comercial':var_mall,
     '¿Ticket hijo?':var_childt,
     'Categoría':var_category_1lv,
     'Subcategoría':var_category_2lv,
     'Tipo disparador':cate_disp,
     'cambia el estado':var_accion_estado,
     '¿Objetivo?':var_target,
     'Asignado':var_assigned,
     'Seguidor asignado 1':var_follower_0,
     'Seguidor asignado 2':var_follower_1,
     'Seguidor asignado 3':var_follower_2,
     'Seguidor asignado 4':var_follower_3,
     'Seguidor asignado 5':var_follower_4,
     'Seguidor asignado 6':var_follower_5
     }),ignore_index=True)



df_tran=df_tran.drop(["conditions", "actions","category_id"],axis=1)
df_tran=pd.concat([df_tran,df_y],axis=1)
df_tran=df_tran.reset_index(drop=True)
